In [1]:
import pickle
import pandas as pd
import numpy as np
import ast
from sklearn.metrics import log_loss
from tqdm import tqdm

In [2]:
train_kaggle=pd.read_csv("../../input/feedback-prize-effectiveness/train.csv")
target='label'
label_mapping={'Ineffective': 0, 'Adequate': 1, 'Effective': 2}
train_kaggle['label'] = train_kaggle['discourse_effectiveness'].map(label_mapping)
train_kaggle

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,label
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,1
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,1
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,1
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,1
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,1
...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,1
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,1
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,1
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,0


In [3]:
span_ids=[0]
s=0
for i in tqdm(range(1,len(train_kaggle))):
    if train_kaggle['essay_id'].iloc[i-1]==train_kaggle['essay_id'].iloc[i]:
        s+=1
        span_ids.append(s)
    else:
        s=0
        span_ids.append(s)
train_kaggle['span_id']=span_ids
train_kaggle

100%|██████████| 36764/36764 [00:00<00:00, 95093.77it/s]


,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,label,span_id
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,1,0
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,1,1
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,1,2
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,1,3
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,1,4
...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,1,0
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,1,1
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,1,2
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,0,3


In [4]:
len(train_kaggle['discourse_id'].unique())

36765

In [5]:
oofs=[]

In [7]:
#tom oofs
df1=pd.read_csv('../../34_RNN2/exp/result/34_v2_02/oof_Tom_768_512_128.csv')
df2=pd.read_csv('../../34_RNN2/exp/result/34_vl_01/oof_Tom_768_512_128.csv')
df3=pd.read_csv('../../999_Stacking/999_v1_22/result/oof_Tom_999_v1_22.csv')
df4=pd.read_csv('../../999_Stacking/999_v1_23/result/oof_Tom_999_v1_23.csv')

df1=train_kaggle[['discourse_id']].merge(df1,how='left',on='discourse_id')
df2=train_kaggle[['discourse_id']].merge(df2,how='left',on='discourse_id')
df3=train_kaggle[['discourse_id']].merge(df3,how='left',on='discourse_id')
df4=train_kaggle[['discourse_id']].merge(df4,how='left',on='discourse_id')

oofs.append(df1[['Ineffective','Adequate','Effective',]].values)
oofs.append(df2[['Ineffective','Adequate','Effective',]].values)
oofs.append(df3[['Ineffective','Adequate','Effective',]].values)
oofs.append(df4[['Ineffective','Adequate','Effective',]].values)

In [9]:
ensemble_predictions=np.stack(oofs)

In [10]:
def ensemble_loss(weights,return_pred=False):
    weights=np.array(weights)
    weights=weights.reshape(-1,1,1)/weights.sum()
    p=weights.reshape(-1,1,1)*ensemble_predictions
    p=p.sum(0)
    loss=log_loss(train_kaggle[target],p)
    if return_pred:
        return loss,p
    else:
        return loss    

In [11]:
from skopt import gp_minimize

results=gp_minimize(ensemble_loss, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                    verbose=True,random_state=2021)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0094
Function value obtained: 0.5716
Current minimum: 0.5716
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0078
Function value obtained: 0.5705
Current minimum: 0.5705
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0075
Function value obtained: 0.5706
Current minimum: 0.5705
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0073
Function value obtained: 0.5717
Current minimum: 0.5705
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0085
Function value obtained: 0.5709
Current minimum: 0.5705
Iteration No: 6 started. Evaluating

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.4250
Function value obtained: 0.5710
Current minimum: 0.5703
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.4955
Function value obtained: 0.5707
Current minimum: 0.5703
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.5584
Function value obtained: 0.5732
Current minimum: 0.5703
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.5629
Function value obtained: 0.5705
Current minimum: 0.5703
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.6760
Function value obtained: 0.5722
Current minimum: 0.5703
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.9988
Function value obtained: 0.5710
Current minimum: 0.5703
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 1.0344
Function value obtained: 0.5704
Current minimum: 0.5703
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.9759
Function value obtained: 0.5716
Current minimum: 0.5703
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.9954
Function value obtained: 0.5703
Current minimum: 0.5703
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 1.0530
Function value obtained: 0.5703
Current minimum: 0.5703
Iteration No: 86 started. Sea

In [12]:
best_weights=np.array(results['x'])/sum(results['x'])
print(best_weights)

[0.23678063 0.42236033 0.14750601 0.19335303]


In [13]:
loss,ensemble_pred=ensemble_loss(best_weights,True)
loss

0.5702756938683863